In [2]:
import sys
import os

sys.path.append('../')

import pandas as pd
import numpy as np
from util import util
import importlib as imp
imp.reload(util)
from util import globalvar as gv
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder

In [3]:
file_name = '../Clean/ready_to_analyse.csv'
dtype_dict = {gv.ORI_STATE: 'category',\
                   gv.DEST_STATE: 'category',  gv.CANCELLED: 'boolean', gv.CANCELLATION_CODE: 'category',\
                    gv.DIVERTED: 'boolean', gv.SCHEDULED_ELAPSED_TIME: 'int64', gv.DISTANCE: 'int64',\
                    gv.DIV_AIRPORT_LANDINGS: 'category',  gv.DIV_REACH_DEST: 'boolean'}

df = pd.read_csv(file_name, dtype=dtype_dict,parse_dates=[gv.FLIGHT_DATE])

df[gv.SCHEDULED_DEP_TIME]=pd.to_datetime(df[gv.SCHEDULED_DEP_TIME],format='%H:%M:%S')
df[gv.SCHEDULED_ARR_TIME]=pd.to_datetime(df[gv.SCHEDULED_ARR_TIME], format='%H:%M:%S')

In [ ]:
df_aa = df[(df[gv.REPORTING_AIRLINE]=='AA')&(df[gv.ARR_DELAY_TIME]>=15)]
X = df_aa[[gv.ORI_CITY, gv.ORI_AIRPORT, gv.ORI_STATE, gv.DEST_CITY, gv.DEST_AIRPORT, gv.DEST_STATE]]
y = df_aa[gv.ARR_DELAY_TIME]


# Manually label encode the categorical variable
label_encoder = LabelEncoder()
for column in X.select_dtypes(include=['object']).columns:
    X[column] = label_encoder.fit_transform(X[column])

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize Linear Regression model
lr = LinearRegression()

# Sequential Forward Selection
sfs = SFS(lr, 
          k_features='best', 
          forward=True, 
          floating=False, 
          scoring='r2',
          cv=0)

# Perform SFS
sfs = sfs.fit(X_train, y_train)

# Selected features
selected_features = X_train.columns[list(sfs.k_feature_idx_)]
print("Selected features:", selected_features)

# Evaluate the model on the test set
lr.fit(X_train[selected_features], y_train)
y_pred = lr.predict(X_test[selected_features])
print("Test set R-squared: ", r2_score(y_test, y_pred))